# Lab | Unsupervised learning intro
### Instructions
It's the moment to perform clustering on the songs you collected. Remember that the ultimate goal of this little project is to improve the recommendations of artists. Clustering the songs will allow the recommendation system to limit the scope of the recommendations to only songs that belong to the same cluster - songs with similar audio features.

The experiments you did with the Spotify API and the Billboard web scraping will allow you to create a pipeline such that when the user enters a song, you:

1. Check whether or not the song is in the Billboard Hot 200.

2. Collect the audio features from the Spotify API.

After that, you want to send the Spotify audio features of the submitted song to the clustering model, which should return a cluster number.

We want to have as many songs as possible to create the clustering model, so we will add the songs you collected to a bigger dataset available on Kaggle containing 160 thousand songs.



# Import Libraries

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import time 

from dotenv import load_dotenv
import os

import warnings
warnings.filterwarnings("ignore")

<div class="alert alert-block alert-info">
<h1>Part I : Preparing Data</h1>
</div>

# Load Data
- This dataset is the audio feature from the previous lab

In [2]:
df = pd.read_csv("audio_features_249.csv")
df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist_id,artist_name
0,0.550,0.750,0,-3.289,1,0.1460,0.05610,0.000000,0.1290,0.381,...,audio_features,5JVA0t7r2Y7m9NaHmgaeiC,spotify:track:5JVA0t7r2Y7m9NaHmgaeiC,https://api.spotify.com/v1/tracks/5JVA0t7r2Y7m...,https://api.spotify.com/v1/audio-analysis/5JVA...,147410,4,Remedy,2NpPlwwDVYR5dIj0F31EcC,Leony
1,0.714,0.425,1,-8.064,1,0.0809,0.40300,0.000105,0.1150,0.316,...,audio_features,0RSZ8EmUPEN3ySfCgytPke,spotify:track:0RSZ8EmUPEN3ySfCgytPke,https://api.spotify.com/v1/tracks/0RSZ8EmUPEN3...,https://api.spotify.com/v1/audio-analysis/0RSZ...,165477,4,Auf & Ab,5ZY4M2aGiTaZQEP6HfqeJc,Montez
2,0.761,0.525,11,-6.900,1,0.0944,0.44000,0.000007,0.0921,0.531,...,audio_features,6CDzDgIUqeDY5g8ujExx2f,spotify:track:6CDzDgIUqeDY5g8ujExx2f,https://api.spotify.com/v1/tracks/6CDzDgIUqeDY...,https://api.spotify.com/v1/audio-analysis/6CDz...,238805,4,Heat Waves,4yvcSjfu4PC0CYQyLy4wSq,Glass Animals
3,0.774,0.792,10,-4.021,1,0.0523,0.05100,0.000000,0.1550,0.507,...,audio_features,3eJH2nAjvNXdmPfBkALiPZ,spotify:track:3eJH2nAjvNXdmPfBkALiPZ,https://api.spotify.com/v1/tracks/3eJH2nAjvNXd...,https://api.spotify.com/v1/audio-analysis/3eJH...,139672,4,Acapulco,07YZf4WDAMNwqr4jfgOZ8y,Jason Derulo
4,0.762,0.766,7,-3.955,1,0.0343,0.00776,0.000070,0.1280,0.442,...,audio_features,5fwSHlTEWpluwOM0Sxnh5k,spotify:track:5fwSHlTEWpluwOM0Sxnh5k,https://api.spotify.com/v1/tracks/5fwSHlTEWplu...,https://api.spotify.com/v1/audio-analysis/5fwS...,287120,4,Pepas,329e4yvIujISKGKz1BZZbO,Farruko


# Get tracks from playlists

- We have retrieved 249 audio features from the previous lab
- Since we want to cluster songs for recommendation, I prefer to get a few more. So I listed 3 playlist from spotify
- In this section, we'll retrieve 150 more songs from 3 playlists

In [3]:
# Get dotenv environment
load_dotenv() # We'll get environment variables in the .env file to use when called 

client_id = os.getenv("client_id")
client_secret = os.getenv("client_secret")

# Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [6]:
def get_audio_features(playlist_id):
    
    """
    This function returns audio features from the given a playlist_id on spotify
    """
    # Get 50 tracks from playlist
    playlist = sp.user_playlist_tracks("spotify", playlist_id)
    
    # Extract track_id, title, artist_id, artist_name
    track_ids = [playlist['items'][i]['track']['id'] for i in range(playlist['total'])]
    title = [playlist['items'][i]['track']['name'] for i in range(playlist['total'])]
    
    artist_id = [playlist['items'][i]['track']['artists'][0]['id'] for i in range(playlist['total'])]
    artist_name = [playlist['items'][i]['track']['artists'][0]['name'] for i in range(playlist['total'])]
    
    # extract the audio features
    audio_features = sp.audio_features(track_ids)
    
    # store audio features in a dataframe
    df = pd.DataFrame(audio_features)
    df['title'] = title
    df['artist_id'] = artist_id
    df['artist_name'] = artist_name
    
    return df

In [7]:
df_results = []

def audio_features_multiple_playlist(playlist_id_list):
    """
    In case you have multiple playlist,
    this function takes an argument of a list of playlist_id &
    return a list of audio features
    """
    for playlist_id in playlist_id_list:
        playlist_id = get_audio_features(playlist_id)
        # df_result = pd.concat([df_result, playlist_id])
        df_results.append(playlist_id)
        time.sleep(10)


In [8]:
# Set up multiple playlists
viral50 = "37i9dQZEVXbLiRSasKsNU9"
top50_daily = "37i9dQZEVXbMDoHDwVN2tF"
top50_weekly = "37i9dQZEVXbNG2KDcFcKOF"
playlist_list = [viral50, top50_daily, top50_weekly]

In [9]:
# Calling function to get audio features from multiple playlists
audio_features_multiple_playlist(playlist_list)

# Turn the result in to one dataframe
df_result = pd.DataFrame()

for m in df_results:
    df_result = df_result.append(m)
    # df = df.append(df_res)

In [10]:
# Concat the dataset we have got from the previous lab together
df = pd.concat([df, df_result]).reset_index(drop=True)

In [11]:
# Check df
print(len(df))
df.head()

399


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist_id,artist_name
0,0.550,0.750,0,-3.289,1,0.1460,0.05610,0.000000,0.1290,0.381,...,audio_features,5JVA0t7r2Y7m9NaHmgaeiC,spotify:track:5JVA0t7r2Y7m9NaHmgaeiC,https://api.spotify.com/v1/tracks/5JVA0t7r2Y7m...,https://api.spotify.com/v1/audio-analysis/5JVA...,147410,4,Remedy,2NpPlwwDVYR5dIj0F31EcC,Leony
1,0.714,0.425,1,-8.064,1,0.0809,0.40300,0.000105,0.1150,0.316,...,audio_features,0RSZ8EmUPEN3ySfCgytPke,spotify:track:0RSZ8EmUPEN3ySfCgytPke,https://api.spotify.com/v1/tracks/0RSZ8EmUPEN3...,https://api.spotify.com/v1/audio-analysis/0RSZ...,165477,4,Auf & Ab,5ZY4M2aGiTaZQEP6HfqeJc,Montez
2,0.761,0.525,11,-6.900,1,0.0944,0.44000,0.000007,0.0921,0.531,...,audio_features,6CDzDgIUqeDY5g8ujExx2f,spotify:track:6CDzDgIUqeDY5g8ujExx2f,https://api.spotify.com/v1/tracks/6CDzDgIUqeDY...,https://api.spotify.com/v1/audio-analysis/6CDz...,238805,4,Heat Waves,4yvcSjfu4PC0CYQyLy4wSq,Glass Animals
3,0.774,0.792,10,-4.021,1,0.0523,0.05100,0.000000,0.1550,0.507,...,audio_features,3eJH2nAjvNXdmPfBkALiPZ,spotify:track:3eJH2nAjvNXdmPfBkALiPZ,https://api.spotify.com/v1/tracks/3eJH2nAjvNXd...,https://api.spotify.com/v1/audio-analysis/3eJH...,139672,4,Acapulco,07YZf4WDAMNwqr4jfgOZ8y,Jason Derulo
4,0.762,0.766,7,-3.955,1,0.0343,0.00776,0.000070,0.1280,0.442,...,audio_features,5fwSHlTEWpluwOM0Sxnh5k,spotify:track:5fwSHlTEWpluwOM0Sxnh5k,https://api.spotify.com/v1/tracks/5fwSHlTEWplu...,https://api.spotify.com/v1/audio-analysis/5fwS...,287120,4,Pepas,329e4yvIujISKGKz1BZZbO,Farruko


In [12]:
# Check number of duplicated
df.duplicated().sum()

51

In [13]:
# Do a quick check on the song that duplicate
# df[df.duplicated()]
df[df['id'].str.contains('6tAKikIvnoWfUeZrfkopLL')] # pick one random duplicated song to see how it duplicates

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist_id,artist_name
220,0.79,0.979,11,-4.637,0,0.117,0.0177,0.000192,0.237,0.898,...,audio_features,6tAKikIvnoWfUeZrfkopLL,spotify:track:6tAKikIvnoWfUeZrfkopLL,https://api.spotify.com/v1/tracks/6tAKikIvnoWf...,https://api.spotify.com/v1/audio-analysis/6tAK...,146087,4,Friesenjung,6CP5wWvO8oIxedESJNCN4H,Ski Aggu
278,0.79,0.979,11,-4.637,0,0.117,0.0177,0.000192,0.237,0.898,...,audio_features,6tAKikIvnoWfUeZrfkopLL,spotify:track:6tAKikIvnoWfUeZrfkopLL,https://api.spotify.com/v1/tracks/6tAKikIvnoWf...,https://api.spotify.com/v1/audio-analysis/6tAK...,146087,4,Friesenjung,6CP5wWvO8oIxedESJNCN4H,Ski Aggu


In [14]:
# Drop duplicated rows
df = df.drop_duplicates().reset_index(drop=True)

In [15]:
# We have total 355 songs with audio_features for clustering
len(df)

348

<div class="alert alert-block alert-info">
<h1>Part II : Clustering the songs: Unsupervised Learning</h1>
</div>

In [28]:
# Import Libraries
from sklearn.preprocessing import StandardScaler
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.metrics import pairwise_distances_argmin_min

# Select the features needed for clustering

In [19]:
# Get list of column names & copy them to create a feature list
df.columns.tolist()

['danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'type',
 'id',
 'uri',
 'track_href',
 'analysis_url',
 'duration_ms',
 'time_signature',
 'title',
 'artist_id',
 'artist_name']

In [20]:
# Now we have 11 features
X = df[['danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo']]

In [34]:
# Check data
X.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.550,0.750,0,-3.289,1,0.1460,0.05610,0.000000,0.1290,0.381,172.124
1,0.714,0.425,1,-8.064,1,0.0809,0.40300,0.000105,0.1150,0.316,99.034
2,0.761,0.525,11,-6.900,1,0.0944,0.44000,0.000007,0.0921,0.531,80.870
3,0.774,0.792,10,-4.021,1,0.0523,0.05100,0.000000,0.1550,0.507,122.062
4,0.762,0.766,7,-3.955,1,0.0343,0.00776,0.000070,0.1280,0.442,130.001


In [37]:
# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X_scaled.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,-1.048702,0.246102,-1.412701,1.321199,0.860233,0.367299,-0.652095,-0.244503,-0.484021,-0.625811,1.675107
1,0.217514,-1.701567,-1.146240,-0.859459,0.860233,-0.326813,0.947521,-0.243684,-0.587904,-0.931292,-1.143060
2,0.580393,-1.102284,1.518366,-0.327881,0.860233,-0.182873,1.118134,-0.244450,-0.757827,0.079147,-1.843418
3,0.680764,0.497801,1.251906,0.986907,0.860233,-0.631753,-0.675612,-0.244503,-0.291095,-0.033646,-0.255158
4,0.588114,0.341987,0.452524,1.017048,0.860233,-0.823673,-0.874999,-0.243959,-0.491441,-0.339128,0.050950
